<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js093_electronQED3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js093_electronQED3D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.12.03 created,  last updated on 2024.03.13
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js093_electronQED3D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js093_electronQED3D
//    Copyright(C) 2018-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2018.03.07 created, last updated on 2018.12.02
//    ver 0.0.1  2019.01.25 v1, last updated on 2021.08.31
//    ver 0.0.2  2021.11.06 v2, last updated on 2021.11.06
//    ver 0.0.3  2023.05.04 v3, last updated on 2023.09.06
//
// -------------------- Quantum Electron Dynamics 3D (QED3D)
//
// - time dependent Schrodinger equation: i(d/dt)psi(r,t) = H psi(r,t)
// - time evolution
//    psi(r,t+dt) = exp(-i dt H) psi(r,t),  (H:Hamiltonian of the system)
//      H = -delta/2 + V(r), delta = d^2/dx^2 + d^2/dy^2 + d^2/dz^2
//    psi(r,t+dt) = exp(-i dt H) psi(r,t) nearly=
//      {exp(-i(dt/2)V} {exp(i dt(delta/2)} {exp(-i(dt/2)V} psi(r,t)
// - algorism: {exp(i dt(delta/2)}
//     QED: Watanabe's algorism (semi-implicit method)
//     Naoki Watanabe, Masaru Tsukada; arXiv:physics/0011068v1
//     (Published from Physical Review E. 62, 2914, (2000).)
//
//     Cayley's form : exp(i dt delta/2) nearly= (1 + i dt delta/4)/(1 - i dt delta/4)
//       psi(r,t+dt) = exp(i dt delta/2) psi(r,dt)
//       (1 - i dt delta/4) psi(r,t+dt) = (1 + i dt delta/4) psi(r,t)
//
//     difference form psi(r,t) --> psi(j,n)
//        psi(j,n+1) - i (dt/dx^2)/4 {psi(j-1,n+1))-2psi(j,n+1)+psi(j+1,n+1)}
//               = psi(j,n) + i (dt/dx^2)/4 {psi(j-1,n))-2psi(j,n)+psi(j+1,n)}
//        x i(4dx^2/dt) by each term
//        psi(j-1,n+1) + A Psi(j,n+1) + psi(j+1,n+1) = -psi(j-1,n) + B Psi(j,n) -psi(j+1,n)
//          where A=(i4dx^2/dt)-2, B=(i4dx^2/dt)+2
//          bnj = -psi(j-1,n) + B Psi(j,n) -psi(j+1,n) is calculated using known psi(j,n)
//          psi(j-1,n+1) + A Psi(j,n+1) + psi(j+1,n+1) = bnj
//
//     solve tri-diagonal equation  A X = B
//         | a1  1  0  0 |  | x1 |    | b1 |
//         |  1 a2  1  0 |  | x2 |  = | b2 |
//         |  0  1 a3  1 |  | x3 |    | b3 |
//         |  0  0  1 a4 |  | x4 |    | b4 |
//
//       u[1] = 1.0/a[1]  // u[] : work vector
//       x[1] = b[1]*u[1]
//
//       for (i=2; i<=N-2; i++) { //forward elimination
//          u[i] = 1/(a[i]-u[i-1])
//          x[i] = (b[i]-x[i-1])*u[i]
//       }
//       for (i=N-3; i<=1; i--) { //backward substitution
//          x[i] -= x[i+1]*u[i]
//       }
//
// --------------------
*/

const electronQED3D = (function(){ // ====================  electronQED3D Module  ====================

	// au: atomic unit hBar=1,e=1,me=1,a0=1
	const g_auLength = 5.29177211e-11;				// (m) 1(au) = g_auLength (m)
	const g_auTime = 2.418884326e-17;				// (s) 1(au) = g_auTime (s)
	const g_auEnergy = 4.35974465e-18;				// (J) 1(au) = g_auEnergy (J)
	const g_au2eV = 27.211386;						// (eV) 1(au) = 27.211386 (eV)
	const g_nMax = 32;								// max(g_NNx,g_NNy,g_NNz)

	let g_NNx = 32;									// g_NNx*g_dx == x-boxsize
	let g_NNy = 32;									// g_NNy*g_dy == y-boxsize
	let g_NNz = 32;									// g_NNz*g_dz == z-boxsize
	let g_sysTime = 0.0;							// (au) system time
	let g_dx = 1.0/2.0;								// (au) x-division
	let g_dy = g_dx;								// (au) y-division
	let g_dz = g_dx;								// (au) z-division
	let g_timeStep = 0.25*(g_dx*g_dx);				// (au) time step dt
	let g_dampingFactor = 0.2*g_dx*g_dx;			// steepest descent loss process

	const g_psi = dim4( g_nMax, g_nMax, g_nMax, 2 );// Re(psi) = psi[i][j][k][0], Im(psi) = psi[i][j][k][1]
	const g_vv = dim3( g_nMax, g_nMax, g_nMax );	// external potential vv[i][j][k]
	const g_wrk = dim4( g_nMax, g_nMax, g_nMax, 2 );// work wave in steepestDescent()
	const g_bRe = dim1( g_nMax );					// work b vector in kxStep(),kyStep(),kzStep()
	const g_bIm = dim1( g_nMax );					// work b vector in kxStep(),kyStep(),kzStep()
	const g_uRe = dim1( g_nMax );					// work u vector in kxStep(),kyStep(),kzStep()
	const g_uIm = dim1( g_nMax );					// work u vector in kxStep(),kyStep(),kzStep()


	function dim1( n ) {
		return new Float64Array( n );
	}

	function dim3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}

	function dim4( ni, nj, nk, nl ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = [];
				for (let k=0; k<nk; k++) {
					a[i][j][k] = new Float64Array( nl );
				}
			}
		}
		return a;
	}


	// --------------------  set theme param.  --------------------

	let g_drawWallFlag = 0;         // draw wall flag
	let g_drawDensMag = 1000.0;     // draw magnitude of psi density
	let g_drawCurrMag = 10000.0;    // draw magnitude of probability current flow

	const g_themeInfo = [
		// 0:theme name             1:potential 2:wave(type,x,y,z,width,px,py,pz)   3:draw(dens,curr,wall)
		['free electron in the box',  0,        [0, 8.0,8.0,8.0, 1.5, 2.0,0.0,0.0], [1000.0,30000.0, 0] ],
		['in paravollic potential',   1,        [0, 8.0,5.0,8.0, 1.0, 2.0,0.0,0.0], [1000.0,10000.0, 0] ],
		['in well potential',         2,        [0, 8.0,8.0,8.0, 1.0, 2.0,0.0,0.0], [1000.0,10000.0, 0] ],
		['in H like V(r)=1/r,Gauss',  3,        [0, 8.0,8.0,8.0, 1.0, 0.3,0.0,0.0], [1000.0,30000.0, 0] ],
		['H like V(r)=1/r,1s-state',  3,        [1, 0],                             [1500.0,30000.0, 0] ],
		['H like V(r)=1/r,2s-state',  3,        [1, 1],                             [1500.0,30000.0, 0] ],
		['H like V(r)=1/r,2px-state', 3,        [1, 2],                             [1500.0,30000.0, 0] ],
		['H like V(r)=1/r,2py-state', 3,        [1, 3],                             [1500.0,30000.0, 0] ],
		['H like V(r)=1/r,2pz-state', 3,        [1, 4],                             [1500.0,30000.0, 0] ],
		['tunnel effect',             4,        [0, 4.0,8.0,8.0, 1.0, 2.0,0.0,0.0], [1500.0,10000.0, 1] ],
		['double slit',               5,        [0, 4.0,8.0,8.0, 1.0, 2.0,0.0,0.0], [1500.0,10000.0, 1] ],
		['scatter sphere',            6,        [0, 4.0,8.0,8.0, 1.0, 2.0,0.0,0.0], [1500.0,30000.0, 1] ],
		['scatter proton',            7,        [0, 4.0,8.0,8.0, 1.0, 2.0,0.0,0.0], [1500.0,30000.0, 1] ],
		[]
	];

	function setThemeParam(theme) {
		g_drawDensMag = g_themeInfo[theme][3][0];
		g_drawCurrMag = g_themeInfo[theme][3][1];
		g_drawWallFlag = g_themeInfo[theme][3][2];
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme ) {

		setPotentialAndWave(theme);
		setThemeParam(theme);
		g_sysTime = 0.0;
	}

	function setPotentialAndWave(theme) {
		const x0=g_NNx*g_dx/2.0, y0=g_NNy*g_dy/2.0, z0=g_NNz*g_dz/2.0;

		const themaPotential = g_themeInfo[theme][1]; // 0:free 1:harmonic 2:H like
		if (themaPotential==0) { // 0:free V = 0 elsewhere
			setBoxPotential();
		} else if (themaPotential==1) { // 1:harmonic
			const harmonick0 = 8.0; // potential at x = g_NNx*g_dx
			setHarmonicPotential(harmonick0);
		} else if (themaPotential==2) { // 2:well
			const r0 = 6.0, vv0 = 8.0;
			setWellPotential(r0,vv0);
		} else if (themaPotential==3) { // 3:H like
			setProtonPotential(x0,y0,z0);
		} else if (themaPotential==4) { // 4:wall (tunnel effect)
			const wallPos = x0, wallHeight = 2.5, wallThick = 1.0;
			setWallPotential(wallPos, wallThick, wallHeight);
		} else if (themaPotential==5) { // 5:double slit
			const wallPos = x0, wallHeight = 8.0, wallThick = 1.0, slitWidth = 1.5, slitSpan = 4.0;
			setSlitPotential(wallPos, wallThick, wallHeight, slitWidth, slitSpan);
		} else if (themaPotential==6) { // 6:sphere
			const xPos = x0+2.0, yPos = y0-2.0, zPos = z0, r0 = 2.5, vHeight = 8.0;
			setSpherePotential(xPos, yPos, zPos, r0, vHeight);
		} else if (themaPotential==7) { // 7:proton
			setProtonPotential(x0+3.0,y0-3.0,z0);
		}

		const themaWaveType = g_themeInfo[theme][2][0]; // 0:gaussian 1:H-state
		if (themaWaveType==0) { // 0:gaussian
			const waveinfo = g_themeInfo[theme][2];
			const xPos = waveinfo[1], yPos = waveinfo[2], zPos = waveinfo[3], waveWidth = waveinfo[4];
			const px = waveinfo[5], py = waveinfo[6], pz = waveinfo[7];
			setGaussianWave(xPos, yPos, zPos, waveWidth, px, py, pz);
		} else if (themaWaveType==1) { // 1:H-state
			const xPos = Math.floor(g_NNx*g_dx/2.0+0.1);
			const yPos = Math.floor(g_NNy*g_dy/2.0+0.1)
			const zPos = Math.floor(g_NNz*g_dz/2.0+0.1)
			const electronState = g_themeInfo[theme][2][1]; // 0:1s 1:2s 2:2px 3:2py 4:2pz 5:3s
			setHOrbitalWave(xPos,yPos,zPos, electronState);
		} else if (themaWaveType==2) { // option
			;
		}
	}

	// --- potential

	function setBoxPotential() { // type 0
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					g_vv[i][j][k] = 0.0;
				}
			}
		}
	}

	function setHarmonicPotential(k0) { // type 1
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, dy=g_dy, dz=g_dz;
		const aa = k0/(nnx*dx*nnx*dx/4);
		const x0 = nnx*dx/2.0, y0 = nny*dy/2.0, z0 = nnz*dz/2.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					const x = i*dx, y = j*dy, z = k*dz;
					g_vv[i][j][k] = aa*((x-x0)*(x-x0)+(y-y0)*(y-y0)+(z-z0)*(z-z0));
				}
			}
		}
	}

	function setWellPotential(r0, v0) { // type 2
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, dy=g_dy, dz=g_dz;
		const x0 = Math.floor(nnx*dx/2.0+0.1), y0 = Math.floor(nny*dy/2.0+0.1), z0 = Math.floor(nnz*dz/2.0+0.1);
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					const x = i*dx, y = j*dy, z = k*dz;
					const r = Math.sqrt((x-x0)*(x-x0)+(y-y0)*(y-y0)+(z-z0)*(z-z0));
					g_vv[i][j][k] = (r<r0) ? 0.0 : v0;
				}
			}
		}
	}

	function setProtonPotential(xPos,yPos,zPos) { // type 3, type 7
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, dy=g_dy, dz=g_dz;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					const x = i*dx, y = j*dy, z = k*dz;
					let r = Math.sqrt((x-xPos)*(x-xPos)+(y-yPos)*(y-yPos)+(z-zPos)*(z-zPos));
					if (r<dx) r = dx/2.0;
					g_vv[i][j][k] = -1.0/r;
				}
			}
		}
	}

	function setWallPotential(xPos, vThick, vHeight) { // type 4
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, dy=g_dy, dz=g_dz;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					const x = i*dx, y = j*dy, z = k*dz;
					g_vv[i][j][k] = (x>=xPos && x<xPos+vThick) ? vHeight : 0.0;
				}
			}
		}
	}

	function setSlitPotential(wallPos, wallThick, wallHeight, slitWidth, slitSpan) { // type 5
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, dy=g_dy, dz=g_dz;
		const ym = nny*dy/2.0, w = slitWidth/2.0, d = slitSpan/2.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					const x = i*dx, y = j*dy, z = k*dz;
					if (x>=wallPos && x<wallPos+wallThick) {
						g_vv[i][j][k] = wallHeight;
						if ((y>=ym-d-w && y<=ym-d+w) || (y>=ym+d-w && y<=ym+d+w)) g_vv[i][j][k] = 0.0;
					} else {
						g_vv[i][j][k] = 0.0;
					}
				}
			}
		}
	}

	function setSpherePotential(xPos, yPos, zPos, r0, vHeight) { // type 6
		const nnx=g_NNx,nny=g_NNy,nnz=g_NNz, dx=g_dx, dy=g_dy, dz=g_dz, sr2=r0*r0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				for (let k=0; k<nnz; k++) {
					const x = i*dx, y = j*dy, z = k*dz;
					const r2 = (x-xPos)*(x-xPos) + (y-yPos)*(y-yPos) + (z-zPos)*(z-zPos);
					g_vv[i][j][k] = (r2 <= sr2 ) ? vHeight : 0.0;
				}
			}
		}
	}


	// --- wave

	function setGaussianWave(xPos, yPos, zPos, wWidth, kx, ky, kz) { // wave type 0
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, dy=g_dy, dz=g_dz;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					const x = i*dx, y = j*dy, z = k*dz;
					const phAb = Math.exp(-((x-xPos)*(x-xPos)+(y-yPos)*(y-yPos)+(z-zPos)*(z-zPos))/(4.0*wWidth*wWidth));
					const phPh = kx*x+ky*y+kz*z;
					g_psi[i][j][k][0] = phAb*Math.cos(phPh);
					g_psi[i][j][k][1] = phAb*Math.sin(phPh);
				}
			}
		}
		setWaveBoundary(g_psi);
		normalize(g_psi);
	}

	function setHOrbitalWave(xPos, yPos, zPos, state) { // wave type 1
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, dy=g_dy, dz=g_dz;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					const x = i*dx, y = j*dy, z = k*dz;
					const r = Math.sqrt((x-xPos)*(x-xPos)+(y-yPos)*(y-yPos)+(z-zPos)*(z-zPos));

					let phAb = 0.0;
					if (state==0) { // 1s
						phAb = Math.exp( -r );
					} else if (state==1) { // 2s
						phAb = (1.0-0.5*r)*Math.exp( -r/2.0 );
					} else if (state==2) { // 2px
						phAb = (x-xPos)*Math.exp( -r/2.0 );
					} else if (state==3) { // 2py
						phAb = (y-yPos)*Math.exp( -r/2.0 );
					} else if (state==4) { // 2pz
						phAb = (z-zPos)*Math.exp( -r/2.0 );
					} else if (state==5) { // 3s
						phAb = (1.0-(2.0/3.0)*r+(2.0/27.0)*r*r)*Math.exp( -r/3.0 );
					} else if (state==6) { // 3px
						phAb = (x-xPos)*(1.0-(1.0/6.0)*r)*Math.exp( -r/3.0 );
					} else if (state==7) { // 3py
						phAb = (y-yPos)*(1.0-(1.0/6.0)*r)*Math.exp( -r/3.0 );
					} else if (state==8) { // 3pz
						phAb = (z-zPos)*(1.0-(1.0/6.0)*r)*Math.exp( -r/3.0 );
					} else if (state==9) { // 3dex
						phAb = (y-yPos)*(z-zPos)*Math.exp( -r/3.0 );
					} else if (state==10) { // 3dey
						phAb = (z-zPos)*(x-xPos)*Math.exp( -r/3.0 );
					} else if (state==11) { // 3dez
						phAb = (x-xPos)*(y-yPos)*Math.exp( -r/3.0 );
					} else if (state==12) { // 3drz
						phAb = (2.0*(z-zPos)*(z-zPos)-(x-xPos)*(x-xPos)-(y-yPos)*(y-yPos))*Math.exp( -r/3.0 );
					} else if (state==13) { // 3drrz
						phAb = ((x-xPos)*(x-xPos)-(y-yPos)*(y-yPos))*Math.exp( -r/3.0 );
					}

					const phPh = 0.0;
					g_psi[i][j][k][0] = phAb*Math.cos(phPh);
					g_psi[i][j][k][1] = phAb*Math.sin(phPh);
				}
			}
		}
		setWaveBoundary(g_psi);
		normalize(g_psi);
	}

	function setWaveBoundary(ph) { // ph[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				ph[i][j][0][0] = 0.0;
				ph[i][j][0][1] = 0.0;
				ph[i][j][nnz-1][0] = 0.0;
				ph[i][j][nnz-1][1] = 0.0;
			}
		}
		for (let i=0; i<nnx; i++) {
			for (let k=0; k<nnz; k++) {
				ph[i][0][k][0] = 0.0;
				ph[i][0][k][1] = 0.0;
				ph[i][nny-1][k][0] = 0.0;
				ph[i][nny-1][k][1] = 0.0;
			}
		}
		for (let j=0; j<nny; j++) {
			for (let k=0; k<nnz; k++) {
				ph[0][j][k][0] = 0.0;
				ph[0][j][k][1] = 0.0;
				ph[nnx-1][j][k][0] = 0.0;
				ph[nnx-1][j][k][1] = 0.0;
			}
		}
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( lossSW ) {

		g_sysTime += g_timeStep;
		phaseStep(g_psi,g_vv,0.5*g_timeStep);
		kxStep(g_psi,g_timeStep);
		kyStep(g_psi,g_timeStep);
		kzStep(g_psi,g_timeStep);
		phaseStep(g_psi,g_vv,0.5*g_timeStep);

		if (lossSW==1) steepestDescent(g_psi,g_vv,g_dampingFactor);
	}

	function kxStep(ph, deltat) { // ph[][][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		const bRe=g_bRe, bIm=g_bIm, uRe=g_uRe, uIm=g_uIm;
		const a = 4.0*g_dy*g_dy/deltat;
		const aaAb = 4.0+a*a;
		for (let k=1; k<nnz-1; k++) {
			for (let j=1; j<nny-1; j++) {
				for (let i=1; i<nnx-1; i++) {
					bRe[i] = 2.0*ph[i][j][k][0]-a*ph[i][j][k][1] - ph[i+1][j][k][0] - ph[i-1][j][k][0];
					bIm[i] = 2.0*ph[i][j][k][1]+a*ph[i][j][k][0] - ph[i+1][j][k][1] - ph[i-1][j][k][1];
				}

				uRe[1] = -2.0/aaAb;
				uIm[1] = -a/aaAb;
				ph[1][j][k][0] = bRe[1]*uRe[1] - bIm[1]*uIm[1];
				ph[1][j][k][1] = bIm[1]*uRe[1] + bRe[1]*uIm[1];
				//forward elimination
				for (let i=2; i<nnx-1; i++) {
					const auAb = (-2.0-uRe[i-1])*(-2.0-uRe[i-1])+(a-uIm[i-1])*(a-uIm[i-1]);
					uRe[i] = (-2.0-uRe[i-1])/auAb;
					uIm[i] = -(a-uIm[i-1])/auAb;
					ph[i][j][k][0] = (bRe[i]-ph[i-1][j][k][0])*uRe[i] - (bIm[i]-ph[i-1][j][k][1])*uIm[i];
					ph[i][j][k][1] = (bRe[i]-ph[i-1][j][k][0])*uIm[i] + (bIm[i]-ph[i-1][j][k][1])*uRe[i];
				}
				//backward substitution
				for (let i=nnx-3; i>=1; i--) {
					ph[i][j][k][0] -= ph[i+1][j][k][0]*uRe[i] - ph[i+1][j][k][1]*uIm[i];
					ph[i][j][k][1] -= ph[i+1][j][k][0]*uIm[i] + ph[i+1][j][k][1]*uRe[i];
				}
			}
		}
	}

	function kyStep(ph, deltat) { // ph[][][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		const bRe=g_bRe, bIm=g_bIm, uRe=g_uRe, uIm=g_uIm;
		const a = 4.0*g_dy*g_dy/deltat;
		const aaAb = 4.0+a*a;
		for (let i=1; i<nnx-1; i++) {
			for (let k=1; k<nnz-1; k++) {
				for (let j=1; j<nny-1; j++) {
					bRe[j] = 2.0*ph[i][j][k][0]-a*ph[i][j][k][1] - ph[i][j+1][k][0] - ph[i][j-1][k][0];
					bIm[j] = 2.0*ph[i][j][k][1]+a*ph[i][j][k][0] - ph[i][j+1][k][1] - ph[i][j-1][k][1];
				}

				uRe[1] = -2.0/aaAb;
				uIm[1] = -a/aaAb;
				ph[i][1][k][0] = bRe[1]*uRe[1] - bIm[1]*uIm[1];
				ph[i][1][k][1] = bIm[1]*uRe[1] + bRe[1]*uIm[1];
				//forward elimination
				for (let j=2; j<nny-1; j++) {
					const auAb = (-2.0-uRe[j-1])*(-2.0-uRe[j-1])+(a-uIm[j-1])*(a-uIm[j-1]);
					uRe[j] = (-2.0-uRe[j-1])/auAb;
					uIm[j] = -(a-uIm[j-1])/auAb;
					ph[i][j][k][0] = (bRe[j]-ph[i][j-1][k][0])*uRe[j] - (bIm[j]-ph[i][j-1][k][1])*uIm[j];
					ph[i][j][k][1] = (bRe[j]-ph[i][j-1][k][0])*uIm[j] + (bIm[j]-ph[i][j-1][k][1])*uRe[j];
				}
				//backward substitution
				for (let j=nny-3; j>=1; j--) {
					ph[i][j][k][0] -= ph[i][j+1][k][0]*uRe[j] - ph[i][j+1][k][1]*uIm[j];
					ph[i][j][k][1] -= ph[i][j+1][k][0]*uIm[j] + ph[i][j+1][k][1]*uRe[j];
				}
			}
		}
	}

	function kzStep(ph, deltat) { // ph[][][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		const bRe=g_bRe, bIm=g_bIm, uRe=g_uRe, uIm=g_uIm;
		const a = 4.0*g_dy*g_dy/deltat;
		const aaAb = 4.0+a*a;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					bRe[k] = 2.0*ph[i][j][k][0]-a*ph[i][j][k][1] - ph[i][j][k+1][0] - ph[i][j][k-1][0];
					bIm[k] = 2.0*ph[i][j][k][1]+a*ph[i][j][k][0] - ph[i][j][k+1][1] - ph[i][j][k-1][1];
				}

				uRe[1] = -2.0/aaAb;
				uIm[1] = -a/aaAb;
				ph[i][j][1][0] = bRe[1]*uRe[1] - bIm[1]*uIm[1];
				ph[i][j][1][1] = bIm[1]*uRe[1] + bRe[1]*uIm[1];
				//forward elimination
				for (let k=2; k<nnz-1; k++) {
					const auAb = (-2.0-uRe[k-1])*(-2.0-uRe[k-1])+(a-uIm[k-1])*(a-uIm[k-1]);
					uRe[k] = (-2.0-uRe[k-1])/auAb;
					uIm[k] = -(a-uIm[k-1])/auAb;
					ph[i][j][k][0] = (bRe[k]-ph[i][j][k-1][0])*uRe[k] - (bIm[k]-ph[i][j][k-1][1])*uIm[k];
					ph[i][j][k][1] = (bRe[k]-ph[i][j][k-1][0])*uIm[k] + (bIm[k]-ph[i][j][k-1][1])*uRe[k];
				}
				//backward substitution
				for (let k=nnz-3; k>=1; k--) {
					ph[i][j][k][0] -= ph[i][j][k+1][0]*uRe[k] - ph[i][j][k+1][1]*uIm[k];
					ph[i][j][k][1] -= ph[i][j][k+1][0]*uIm[k] + ph[i][j][k+1][1]*uRe[k];
				}
			}
		}
	}

	function phaseStep(ph, vv, deltat) { // ph[][][][], vv[][][]
		const nnx=g_NNx, nny=g_NNy,nnz=g_NNz;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					const th = deltat*vv[i][j][k];
					const costh = Math.cos(th), sinth = Math.sin(th);
					const phRe = ph[i][j][k][0];
					const phIm = ph[i][j][k][1];
					ph[i][j][k][0] = costh*phRe + sinth*phIm;
					ph[i][j][k][1] =-sinth*phRe + costh*phIm;
				}
			}
		}
	}

	// ---- set loss

	function steepestDescent(ph, vv, damp) { // ph[][][][], vv[][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, h2=2.0*g_dx*g_dx;
		const wrk=g_wrk;

		const ee = meanKinetic(ph) + meanPotential(ph,vv);
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					wrk[i][j][k][0] = -(ph[i+1][j][k][0]+ph[i-1][j][k][0]+ph[i][j+1][k][0]+ph[i][j-1][k][0]
						+ph[i][j][k+1][0]+ph[i][j][k-1][0]-6.0*ph[i][j][k][0])/h2+(vv[i][j][k]-ee)*ph[i][j][k][0];
					wrk[i][j][k][1] = -(ph[i+1][j][k][1]+ph[i-1][j][k][1]+ph[i][j+1][k][1]+ph[i][j-1][k][1]
						+ph[i][j][k+1][1]+ph[i][j][k-1][1]-6.0*ph[i][j][k][1])/h2+(vv[i][j][k]-ee)*ph[i][j][k][1];
				}
			}
		}
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					ph[i][j][k][0] -= damp*wrk[i][j][k][0];
					ph[i][j][k][1] -= damp*wrk[i][j][k][1];
				}
			}
		}
		normalize(ph);
	}


	// -------------------- utility

	function norm(ph) { // ph[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		let p = 0.0;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					p += (ph[i][j][k][0]*ph[i][j][k][0]+ph[i][j][k][1]*ph[i][j][k][1]);
				}
			}
		}
		return p*g_dx*g_dy*g_dz;
	}

	function normalize(ph) { // ph[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		const  a = 1.0/Math.sqrt(norm(ph));
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					ph[i][j][k][0] *= a;
					ph[i][j][k][1] *= a;
				}
			}
		}
	}

	function meanPotential(ph, vv) { // ph[][][][], vv[][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		let p=0.0;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					p += vv[i][j][k]*(ph[i][j][k][0]*ph[i][j][k][0]+ph[i][j][k][1]*ph[i][j][k][1]);
				}
			}
		}
		return p*g_dx*g_dy*g_dz;
	}

	function meanKinetic(ph) { // ph[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, h2=g_dx*g_dx;
		let p=0.0;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					const d2phRe = (ph[i+1][j][k][0]+ph[i-1][j][k][0]+ph[i][j+1][k][0]+ph[i][j-1][k][0]+ph[i][j][k+1][0]
						+ph[i][j][k-1][0]-6.0*ph[i][j][k][0])/h2;
					const d2phIm = (ph[i+1][j][k][1]+ph[i-1][j][k][1]+ph[i][j+1][k][1]+ph[i][j-1][k][1]+ph[i][j][k+1][1]
						+ph[i][j][k-1][1]-6.0*ph[i][j][k][1])/h2;
					p += (ph[i][j][k][0]*d2phRe+ph[i][j][k][1]*d2phIm);
				}
			}
		}
		return -0.5*p*g_dx*g_dy*g_dz;
	}

	function probabilityRegion(ph, xBegin, xEnd) { // ph[][][][]
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz;
		let p=0.0;
		for (let i=0; i<nnx; i++) {
			const x = i*g_dx;
			if (x>=xBegin && x<xEnd) {
				for (let j=0; j<nny; j++) {
					for (let k=0; k<nnz; k++) {
						p += (ph[i][j][k][0]*ph[i][j][k][0]+ph[i][j][k][1]*ph[i][j][k][1]);
					}
				}
			}
		}
		return p*g_dx*g_dy*g_dz;
	}

	function psiDensity(i,j,k) {
		return (g_psi[i][j][k][0]*g_psi[i][j][k][0]+g_psi[i][j][k][1]*g_psi[i][j][k][1]);
	}

	function psiPhase(i,j,k) { // 0...2PI
		return (Math.PI + Math.atan2(g_psi[i][j][k][1],g_psi[i][j][k][0])); // atan2(y,x)
	}

	function psiXCurrent(i,j,k) {
		const pRe = (g_psi[i+1][j][k][1]-g_psi[i-1][j][k][1])/(2*g_dx);
		const  pIm = (-g_psi[i+1][j][k][0]+g_psi[i-1][j][k][0])/(2*g_dx);
		return (g_psi[i][j][k][0]*pRe + g_psi[i][j][k][1]*pIm)*g_dx*g_dy*g_dz;
	}

	function psiYCurrent(i,j,k) {
		const pRe = (g_psi[i][j+1][k][1]-g_psi[i][j-1][k][1])/(2*g_dy);
		const pIm = (-g_psi[i][j+1][k][0]+g_psi[i][j-1][k][0])/(2*g_dy);
		return (g_psi[i][j][k][0]*pRe + g_psi[i][j][k][1]*pIm)*g_dx*g_dy*g_dz;
	}

	function psiZCurrent(i,j,k) {
		const pRe = (g_psi[i][j][k+1][1]-g_psi[i][j][k-1][1])/(2*g_dz);
		const pIm = (-g_psi[i][j][k+1][0]+g_psi[i][j][k-1][0])/(2*g_dz);
		return (g_psi[i][j][k][0]*pRe + g_psi[i][j][k][1]*pIm)*g_dx*g_dy*g_dz;
	}

	function psiRe(i,j,k) {
		return g_psi[i][j][k][0];
	}

	function psiIm(i,j,k) {
		return g_psi[i][j][k][1];
	}

	function Vext(i,j,k) {
		return g_vv[i][j][k];
	}


	// --------------------  cloud  --------------------

	const g_cloud = dim3(g_NNx,g_NNy,g_NNz);
	const g_srnd = [];

	function setCloud() {
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dv=g_dx*g_dy*g_dz;

		set_srnd();

		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					g_cloud[i][j][k] = 0;
				}
			}
		}
		let s = 0, ip = 0
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				for (let k=1; k<nnz-1; k++) {
					const dens = psiDensity(i,j,k);
					s += dens*dv;
					while (s>g_srnd[ip] && ip<1000) {
						g_cloud[i][j][k] += 1;
						ip += 1;
					}
				}
			}
		}
	}

	function set_srnd() {
		g_srnd[0] = Math.random();
		for (let i=1; i<1001; i++) {
			g_srnd[i] = g_srnd[i-1] + Math.random();
		}
		for (let i=0; i<1000; i++) {
			g_srnd[i] = g_srnd[i]/g_srnd[1000];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme )
		evolve:			timeEvolution,			// timeEvolution( lossSW )

		setCloud:		setCloud,				// setCloud()

		getAUinSI:		function() { return [ g_auLength, g_auTime, g_auEnergy, g_au2eV ]; },
		getSysParam:	function() { return [ g_NNx, g_NNy, g_NNz, g_dx, g_timeStep ]; },
		getDrawParam:	function(theme) { return g_themeInfo[theme][3]; }, // [ densMag, currMag, wallSW ]
		getNow:			function() { return [ g_sysTime, norm(g_psi), meanKinetic(g_psi), meanPotential(g_psi, g_vv) ]; },
		getPsiDensity:	psiDensity,				// psiDensity(i,j,k)
		getPsiPhase:	psiPhase,				// psiPhase(i,j,k)
		getPsiCurrent:	function(i,j,k) { return [ psiXCurrent(i,j,k), psiYCurrent(i,j,k), psiZCurrent(i,j,k) ]; },
		getPsi:			function(i,j,k) { return [ g_psi[i][j][k][0], g_psi[i][j][k][1] ]; },
		getVext:		function(i,j,k) { return g_vv[i][j][k]; },
		getCloud:		function(i,j,k) { return g_cloud[i][j][k]; },
	};

})(); // ====================  electronQED3D end  ====================


const js093 = (function(){ // ====================  js Module  ====================

	const theModule = electronQED3D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;

	let v_theme = 1;			// 0:free space, 1:parabolic potential, 2:tunnel effect 3:step hill 4:step higher hill
	let v_lossSW = 0;			// if lossSW==1, call lossEnergy()

	let p_NNx, p_NNy, p_NNz, p_dx, p_timeStep; // = theModule.getSysParam();

  let sysTime, norm, kineticEnergy, potentialEnergy;
	let nowData = [];
  let vextArray = [];
  let psiReArray = [];
  let psiImArray = [];

	let dispMode = 1;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	let inStepFlag = false;

  let breakFlag = false;
  let getFieldFlag = true;
  let fieldKind = 1;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();
		viewHome();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme );
			[ p_NNx, p_NNy, p_NNz, p_dx, p_timeStep ] = theModule.getSysParam();
			v_lossSW = 0;
			document.getElementById("slct_lossSW").selectedIndex = 0;
			// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
			g3d.init( p_NNx, p_NNy, p_NNz, p_dx, xCanvasSize, yCanvasSize, 300, 20 );
      getFieldFlag = true;
      fieldKind = 1;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_lossSW );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_lossSW );
			inStepFlag = true;
		}

		draw( ctx, dispMode );

    if ( getFieldFlag ) setFieldData( fieldKind );

		requestAnimationFrame(animate);
	}

  function setFieldData( fieldKind ) {
    if (fieldKind==1) {
			vextArray = [];
			psiReArray = [];
			psiImArray = [];
			nowData = [ sysTime, norm, kineticEnergy, potentialEnergy ];
      for (let i=0; i<p_NNx; i++) {
        vextArray[i] = [];
        psiReArray[i] = [];
        psiImArray[i] = [];
        for (let j=0; j<p_NNy; j++) {
          vextArray[i][j] = [];
          psiReArray[i][j] = [];
          psiImArray[i][j] = [];
          for (let k=0; k<p_NNz; k++) {
            vextArray[i][j][k] = theModule.getVext(i,j,k);
            let pre, pim;
            [ pre, pim ] = theModule.getPsi(i,j,k);
            psiReArray[i][j][k] = pre;
            psiImArray[i][j][k] = pim;
          }
        }
      }
    }
  }


	// --------------------  draw  --------------------

	const gColor = { orb:"#dddd00", dens:"#dd88dd",
					Vext:"#00dd00", Veff:"#0088ff", Vh:"#4444ff", Vxc:"#8800ff", Vx:"#aa00ff",Vc:"#8888ff" };

	function draw( ctx, dispMode ) {
		const kc = p_NNz/2;

		let auLength, auTime, auEnergy, au2eV;
		[ auLength, auTime, auEnergy, au2eV ] = theModule.getAUinSI();
		[ sysTime, norm, kineticEnergy, potentialEnergy ] = theModule.getNow();
		let densMag, currMag, wallSW;
		[ densMag, currMag, wallSW ] = theModule.getDrawParam(v_theme);

		const densFunc = function(i,j,k) { return densMag*theModule.getPsiDensity(i,j,k); };

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);


		if (dispMode==0) {
			dispText( "3D psi density |psi(x,y,z)|^2" );
			// drawField3D(ctx, rotAngle, fieldFunc, drawMode, wall );
			// drawMode:  0:density, 1:current, 2:phase
			// wall: 0:not draw wall, 1:draw wall
			drawField3D( ctx, 0.0, densFunc, 0, wallSW );

		} else if (dispMode==1) {
			dispText( "3D phase(x,y,z) (angle:Im(psi)/Re(psi))" );
			const phaseFunc = function(i,j,k) { return theModule.getPsiPhase(i,j,k)*radian; };
			// drawField3D(ctx, rotAngle, fieldFunc, drawMode, wall );
			drawField3D( ctx, 0.0, densFunc, 2, wallSW );

		} else if (dispMode==2) {
			dispText( "3D probability current(x,y,z)" );
			// drawField3D(ctx, rotAngle, fieldFunc, drawMode, wall, scale, xPos, yPos );
			drawField3D( ctx, 0.0, densFunc, 1, wallSW );

		} else if (dispMode==3) {
			dispText( "3D cloud <-- |psi(x,y,z)|^2" );
			if ( !pauseFlag || inStepFlag ) theModule.setCloud();
			inStepFlag = false;
			const cloudFunc = function(i,j,k) { return 2.0*theModule.getCloud(i,j,k); };
			// drawField3D(ctx, rotAngle, fieldFunc, drawMode, wall );
			drawField3D( ctx, 0.0, cloudFunc, 0, wallSW );

		} else if (dispMode==4) {
			dispText( "grid2d Re[psi(x,y,0)]" );
			const densXYFunc = function(i,j) { return 20.0*theModule.getPsi(i,j,kc)[0]; };
			// drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			drawGrid2D( ctx, 0.0, densXYFunc, 0.5, 1 );

		} else if (dispMode==5) {
			dispText( "grid2d density(x,y,0)" );
			const zFunc = function(i,j) { return 50.0*theModule.getPsiDensity(i,j,kc) };
			// drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			drawGrid2D( ctx, 0.0, zFunc, 0.5, 1 );

		} else if (dispMode==6) {
			dispText( "grid2d (phase+Vext)(x,y,0)" );
			const zFunc = function(i,j) { return 50.0*theModule.getPsiDensity(i,j,kc) + 0.5*theModule.getVext(i,j,kc) };
			const colorFunc = function(i,j) {
				const z = 40.0*theModule.getPsiDensity(i,j,kc);
				const th = theModule.getPsiPhase(i,j,kc)*180.0/Math.PI;
				return ( Math.abs(z/g3d.cz0)>0.005 ) ? "hsl("+(th)+",100%,50%)" : "#446644";
			}
			// drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc, scale, xPos, yPos [, showBox] )
			drawGrid2D( ctx, 0.0, zFunc, colorFunc, 1 );

		} else if (dispMode==7) { // 3D view - Vext(x,y)
			dispText( "grid2d external potential: Vext(x,y,0)" );
			const zFunc = function(i,j) { return 0.5*theModule.getVext(i,j,kc); };
			// drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			drawGrid2D( ctx, 0.0, zFunc, gColor.Vext, 1 );
		}

		ctx.fillStyle = "#888888";
		ctx.fillText(`box = ${p_NNx*p_dx} x ${p_NNy*p_dx} x ${p_NNz*p_dx} (au)`, 20, yCanvasSize-30);
		ctx.fillText(`norm = ${norm.toFixed(16)}`, 240, yCanvasSize-30);
		ctx.fillText(`time = ${sysTime.toFixed(2)}`, 20, yCanvasSize-10);
		ctx.fillText(`Energy = ${(kineticEnergy+potentialEnergy).toFixed(4)}`, 240, yCanvasSize-10);

		function dispText( str ) {
			ctx.fillStyle = "#888888";
			ctx.fillText( str, 20, yCanvasSize-50 );
		}
	}


	// function drawField3D(ctx, rotAngle, fieldFunc, drawMode, wall );
	// drawMode:  0:density, 1:current, 2:phase
	// wall: 0:not draw wall, 1:draw wall
	function drawField3D(ctx, rotAngle, fieldFunc, drawMode, wall ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, boxSize=300.0, hh=boxSize/g_NNx,zoom=g3d.zoom;
		const cmag = drawField3D.currentMag, radian = 180.0/Math.PI;
		const threshold = drawField3D.threshold;

		g3d.set3DRotateXY(rotAngle);
		g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let ii=0; ii<nnx; ii++) {
			let i=ii; if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj; if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				for (let kk=0; kk<nnz; kk++) {
					let k=kk; if (g3d.pzApex[4]-g3d.pzApex[0]<0) k=nnz-kk-1;

					if ( wall==1 ) {
						const vv = theModule.getVext(i,nny-j-1,k);
						if ( Math.abs(vv)>1.0 ) {
							const r = 0.4*hh, a = Math.min(Math.abs(vv)*0.1,0.8);
							const color = (vv>0) ? `rgba(0,80,0,${a}` : `rgba(120,0,120,${a}`;
							g3d.drawRotatedDisc( ctx, i*dx, j*dx, k*dx, r*zoom, color, sc, xp, yp );
						}
					}

					const f = fieldFunc(i,nny-j-1,k);
					const r = Math.min(Math.pow(Math.abs(f),0.333),0.9*hh);
					if (r>=threshold) {
						let hue;
						if (drawMode<=1) { // hue <-- density ( drawMode  0:density, 1:current )
							hue = 120.0-30.0*r*Math.sign(f);
							if (hue>300.0) hue = 300.0;
							if (hue<-60.0) hue = -60.0;
							hue =( hue+360.0)%360;
						} else if ( drawMode==2 ) { // hue <-- phase
							hue = theModule.getPsiPhase(i,nny-j-1,k)*radian;
						}
						const color = `hsl(${hue},100%,50%)`;
						g3d.drawRotatedDisc( ctx, i*dx, j*dx, k*dx, r*zoom, color, sc, xp, yp );
						if ( drawMode==1 ) {
							const x=i*dx, y=j*dx, z=k*dx;
							let cx, cy, cz;
							[ cx, cy, cz ] = theModule.getPsiCurrent( i, nny-j-1, k );
							const lineColor = (cx>0) ? "#0066ff" : "#ff2222";
							g3d.drawRotatedLine(ctx, x, y, z, x+cx*cmag, y-cy*cmag, z+cz*cmag, lineColor, sc, xp, yp );
						}

					}

				}
			}
		}
		g3d.plotNearEdge( ctx, sc, xp, yp, "#999900"); //yellow
		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText("Ax="+(g3d.Ax*180/Math.PI).toFixed(1)+", Ay="+(g3d.Ay*180/Math.PI).toFixed(1), 10, 15);
		ctx.font = "16px 'sans-serif'";
	}
	drawField3D.threshold = 0.7;
	drawField3D.currentMag = 1000.0;


	// drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
	function drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx, nny = g_NNy, threshold = drawGrid2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( showBox || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"rgb(60,60,0)"); // dark yellow
		for (let jj=0; jj<nny; jj+=inc) {
			let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
			for (let ii=0; ii<nnx; ii++) {
				let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
				if (i<0 || i+1>nnx-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i+1,nny-j-1);
				const x1 = (i+1)*g_dx, y1 = j*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? "hsl("+(th)+",100%,50%)" : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				if (j<0 || j+1>nny-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i,nny-j-2);
				const x1 = i*g_dx, y1 = (j+1)*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? "hsl("+(th)+",100%,50%)" : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( showBox || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"rgb(160,160,0)"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "rgb(80,80,80)";
		ctx.fillText("Ax="+(g3d.Ax*180/Math.PI).toFixed(1)+", Ay="+(g3d.Ay*180/Math.PI).toFixed(1), 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	drawGrid2D.threshold = 0.005


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {  // select theme
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setLossSW() {  // select lossSW
		v_lossSW = 0 + document.getElementById("slct_lossSW").selectedIndex;
	}

	function setDispMode() {  // select dispMode
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle(10,-15);
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, norm, kineticEnergy, potentialEnergy ];
  }

  function pygetFieldData() {
		fieldKind = 0;
    return [ nowData, vextArray, psiReArray, psiImArray ];
  }


	// --------------------  public  --------------------

	return {
		main:				main,				// main()

		reset:				reset,				// reset()
		pause:				pause,				// pause()
		step:				step,				// step()

		setTheme:			setTheme,			// setTheme()
		setLossSW:			setLossSW,			// setLossSW()
		setDispMode:		setDispMode,		// setDispMode()
		viewHome:			viewHome,			// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, norm, kineticEnergy, potentialEnergy ]
    pygetFieldData: pygetFieldData, // ygetFieldData() : return [ nowData, vextArray, psiReArray, psiImArray ]
	};

})(); // ====================  js093 end  ====================


const js = js093;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js093] electron - Quantum Electron Dynamics QED3D</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="480" HEIGHT="480"></canvas>
<br>

<label>theme:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>an electron in the box</option>
<option selected>an electron in parabollic V(r)=k r^2</option>
<option>an electron in the sphere r=6au</option>
<option>H like - Gaussian</option><option>H like - 1s</option><option>H like - 2s</option>
<option>H like - 2px</option><option>H like - 2py</option><option>H like - 2pz</option>
<option>tunnel effect</option><option>double slit</option>
<option>scattering an electron with the sphere</option>
<option>scattering an electron with a proton</option>
</select>
    <span style="margin-right: 20px;"></span>
<label>loss:</label>
<select id="slct_lossSW" onChange="js.setLossSW()">
<option selected>off</option><option>on</option>
</select>
    <span style="margin-right: 20px;"></span>
<button onClick="js.reset()">reset</button>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>density = psi^2</option><option selected>phase (angle:Im(psi)/Re(psi))</option>
<option>probability current</option><option>electron cloud</option>
<option>2D Re( psi(x,y,0) )</option><option>2D psi density(x,y,0)</option>
<option>2D phase(x,y,0)+V(x,y,0)</option>
<option>2D potential V(x,y,0)</option>
</select>
    <span style="margin-right: 100px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ sysTime, norm, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  totalEnergy = kineticEnergy + potentialEnergy
  print(f'i = {i:>2},  iter count = {sysTime:>6.2f} (au),  total energy = {totalEnergy:>10.6f} (au)')
  time.sleep(1)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# change theme, dispMode

import time

themeList = [
  '0: an electron in the box', '1: an electron in parabollic V(r)=k r^2', '2: an electron in the sphere r=6au',
  '3: H like - Gaussian', '4: H like - 1s', '5: H like - 2s', '6: H like - 2px', '7: H like - 2py', '8: H like - 2pz',
  '9: tunnel effect', '10: double slit', '11: scattering an electron with the sphere','12: scattering an electron with a proton' ]

dispModeList = [
  '0: density = psi^2', '1: phase (angle:Im(psi)/Re(psi))', '2: probability current', '3: electron cloud',
  '4: 2D Re( psi(x,y,0) )', '5: 2D psi density(x,y,0)', '6: 2D phase(x,y,0)+V(x,y,0)', '7: 2D potential V(x,y,0)' ]

# exec html-js code
exec_html_js()
print("-- start --")

# reset
eval_js( 'js.reset()' )
print("-- reset --")

# change theme
for theme in [ 1, 4, 9, 10, 11 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print( "-- theme:", themeList[theme], "--" )
  for dispMode in [ 0, 1, 6 ]:
    # change dispMode
    eval_js( 'js.pysetDispMode({})'.format(dispMode) )
    print( "  -- dispMode:",dispModeList[dispMode], "--" )
    time.sleep(1)
    # get data and print
    [ sysTime, norm, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
    totalEnergy = kineticEnergy + potentialEnergy
    print(f'\t iter count = {sysTime:>6.2f} (au),  total energy = {totalEnergy:>10.6f} (au)')
  print("")

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# get field data and convert to numpy array

import time
import numpy as np

# exec html-js code
exec_html_js()
print("-- start --")

# wait a moment
time.sleep(3)

# get field data
print("-- get Vext, psiRe, psiIm array data --")
print("-- wait a minute --  getting data ")
[ nowData, vext, psiRe, psiIm ] = eval_js( 'js.pygetFieldData()' )
[ sysTime, norm, kineticEnergy, potentialEnergy ] = nowData
totalEnergy = kineticEnergy + potentialEnergy
print(f'\t got iter count = {sysTime:>6.2f} (au),  K = {kineticEnergy},  U = {potentialEnergy},  total energy(K+U) = {totalEnergy:>10.6f} (au)')

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- simulator stop --")
print("")

# convert to nampy array
print("-- convert to numpy array Dens --")
Vext = np.array(vext)
PsiRe = np.array(psiRe)
PsiIm =np.array(psiIm)
print(f"Vext shape :{Vext.shape}, PsiRe shape :{PsiRe.shape}, PsiIm shape :{PsiIm.shape} ")

In [ ]:
# save field data

import numpy as np

# pack data
packed_data = np.array([ vext, psiRe, psiIm ])
print("-- packed_data shape :", packed_data.shape, " --")

# save
print("-- save packed_data as 'js093_data.npy' --")
np.save( 'js093_data.npy', packed_data )

In [ ]:
# Vext(x,y,0) / image plot

import numpy as np
import matplotlib.pyplot as plt

def image_plot2d(fld,titleStr):
  nx, ny, nz = fld.shape

  Z = fld[ : , : , int(nz/2)].T
  img = plt.imshow(Z, origin='lower', cmap='jet' )
  plt.colorbar(img)
  plt.title( titleStr )
  plt.show()

image_plot2d(Vext,"Vext(x,y,0)")

In [ ]:
image_plot2d(PsiRe,"real part of psi(x,y,0)")

In [ ]:
image_plot2d(PsiIm,"imaginary part of psi(x,y,0)")

In [ ]:
# density(x,y,z)

import numpy as np

# density
Dens = PsiRe**2 + PsiIm**2

# Dens(x,y,0) image

image_plot2d(Dens,"electron density(x,y,0)")

In [ ]:
# phase(x,y,z)
import numpy as np

# phase
Phase = np.arctan2(PsiIm,PsiRe)*Dens

# Dens(x,y,0) image

image_plot2d(Phase,"phase(x,y,0)")

In [ ]:
# real part of psi(x,y,z) - scatter plot 3D

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

nx, ny, nz = PsiRe.shape
h = 0.5
x = np.linspace(-h*nx, h*nx, nx)
y = np.linspace(-h*ny, h*ny, ny)
z = np.linspace(-h*nz, h*nz, nz)
xx, yy, zz = np.meshgrid(x, y, z)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(yy, xx, zz, c=PsiRe, cmap='jet', s=np.abs(PsiRe)*10)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.title("real part of psi(x,y,z)")
plt.show()

In [ ]:
# real part of psi(x,y,z) - scatter plot 3D / plotly version

import numpy as np
import plotly.graph_objects as go

# prepare numpy array
nx, ny, nz = PsiRe.shape
h = 0.5
x = np.linspace(-h*nx, h*nx, nx)
y = np.linspace(-h*ny, h*ny, ny)
z = np.linspace(-h*nz, h*nz, nz)
xx, yy, zz = np.meshgrid(x, y, z)

# plot
fig = go.Figure(data=go.Scatter3d(
    x=yy.flatten(), y=xx.flatten(), z=zz.flatten(),
    mode='markers',
    marker=dict(
        size=np.sqrt(abs(PsiRe.flatten())) * 20,
        color=PsiRe.flatten(),
        colorscale='Jet',
        opacity=0.3
      )
    ))

fig.update_layout(
    title="real part of psi(x,y,z)",
    width=800, height=800,
    scene={
        'camera_eye': {"x": 1, "y": -1.5, "z": 0.5},
      },
    )

fig.show()